In [29]:
#imports
from openai import AzureOpenAI
import pandas as pd
import pickle as pkl
import json
import re
import hashlib
import threading
import concurrent.futures
from ast import *
import string

import openai
from config import api_key
from utils import *

pd.set_option('display.max_columns', None)

In [3]:
# Parameters
round_1_output_path = "../round_1_output/course_skill_pl_final_r1.csv"

sfw_raw_data_path = "../input_data/Copy of Sfw_dataset-2023-10 (002).xlsx"
sfw_raw_data_sheet = "TSC_CCS_K&A"
target_sector = "Precision Engineering"

course_raw_data_path = "../input_data/PE TSC_Skills Heatmap_4 Apr 2024.xlsx"
course_raw_data_sheet = "2k course listing"
course_raw_data_cols = ["Unnamed: 7", "TSC Category.1", "TSC/Skill", "No. of WSQ Courses", "No. of Non-WSQ Courses", "Total No. of Courses"]

course_descr_data_path = "../input_data/precision_engineering_courses.xlsx"

checkpoint_file_path = "./checkpoint_files/arc_chart_ref_checkpoint_25062024.txt"
pickle_file_path = "../round_2_output/gpt_output_25062024.pkl"

final_output_path = "../round_2_output/course_skill_pl_arc_chart_full_28062024.csv"

# LLM Configuration

In [4]:
def get_gpt_completion(sys_msg, model="gpt-4o", temperature=0.1):
    client = openai.OpenAI(api_key=api_key,base_url="https://ai-api.analytics.gov.sg")
    response = client.chat.completions.create(
        model=model, 
        messages = sys_msg,
        response_format={"type": "json_object"},
        seed=6800,
        temperature=temperature
    )
    completion_output = literal_eval(response.choices[0].message.content)
    return completion_output

In [4]:
# sys_msg = [{
#     "role": "user",
#     "content":"What is the colour of the sky? Reply me in JSON format {'sky_colour': 'colour'}."}]

In [5]:
# test_result = get_gpt_completion(sys_msg=sys_msg)

In [6]:
# test_result

{'sky_colour': 'blue'}

# Load in TSC RAC Chart

In [5]:
from skill_rac_chart import skill_proficiency_level_details

In [6]:
data = pd.read_csv(round_1_output_path, low_memory=False)

In [7]:
data["course_text"] = data['Course Title'] + " |: " + data["About This Course"] + " | " + data["What You'll Learn"]
data['unique_text'] = data['course_text'] + data['TSC Title/Skill']
data['unique_id'] = data['unique_text'].apply(lambda x: generate_hash(x))

In [8]:
target_df = data[data.proficiency_level==0]
target_df.head(2)

,Course Title,Course Reference Number,TP Name,TP UEN,Programme Type,About This Course,What You'll Learn,TSC Category,TSC Title/Skill,Training Provider,WSQ/Non-WSQ,Appears in 11K list?,proficiency_level,reason,confidence,course_text,unique_text,unique_id
0,Food Safety Course Level 1,TGS-2018502008,COLD STORAGE SINGAPORE (1983) PTE LTD,194700005R,SSG-WSQ,"On completion, participants will have the skil...",1. Practise Good Food Hygiene_x000D_\n2. Use S...,Manufacturing and Operations,Cleanliness and Contamination Control,COLD STORAGE SINGAPORE (1983) PTE LTD,SSG-WSQ,N,0,Course covers food safety which is distinct fr...,high,"Food Safety Course Level 1 |: On completion, p...","Food Safety Course Level 1 |: On completion, p...",13ff8cfaf8d70c1f4d585586a1809f6d310840f8fc5511...
2,Food Safety Course Level 1,TGS-2019504792,GOODWOOD PARK HOTEL PRIVATE LIMITED,194700135H,SSG-WSQ,To equip food handlers with the knowledge and ...,1. Practice good personal hygiene_x000D_\n2. U...,Workplace Safety and Health Management,Workplace Safety and Health System Management,GOODWOOD PARK HOTEL PRIVATE LIMITED,SSG-WSQ,N,0,"The course is related to food safety, not Work...",high,Food Safety Course Level 1 |: To equip food ha...,Food Safety Course Level 1 |: To equip food ha...,fb2a69f7b9e72b1513b40ac673a454a5950d60f3a4b263...


# Pre-processing (PE Knowledge Base)

In [9]:
# Read in skill-level data
sfw_raw_data = pd.read_excel(sfw_raw_data_path, sheet_name=sfw_raw_data_sheet)

In [10]:
# Filter skill-level data to only Precision Engineering
pe_knowledge_df = sfw_raw_data[sfw_raw_data["Sector"] == target_sector]
pe_knowledge_df = pe_knowledge_df.reset_index(drop=True)

In [11]:
pe_knowledge_df.head(2)

,TSC_CCS_Type,TSC_CCS Code,Sector,TSC_CCS Category,TSC_CCS Title,TSC_CCS Description,Proficiency Level,Proficiency Description,Knowledge / Ability Classification,Knowledge / Ability Items
0,tsc,PRE-RAO-1002-1.1,Precision Engineering,Automation Management,Automated Operation Monitoring,Ensure smooth automation operations by maintai...,1,Operate automated robots and system to monitor...,knowledge,Operation terminologies
1,tsc,PRE-RAO-1002-1.1,Precision Engineering,Automation Management,Automated Operation Monitoring,Ensure smooth automation operations by maintai...,1,Operate automated robots and system to monitor...,knowledge,Functions of robots and smart automated tools


In [12]:
pe_knowledge_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3897 entries, 0 to 3896
Data columns (total 10 columns):
 #   Column                              Non-Null Count  Dtype 
---  ------                              --------------  ----- 
 0   TSC_CCS_Type                        3897 non-null   object
 1   TSC_CCS Code                        3897 non-null   object
 2   Sector                              3897 non-null   object
 3   TSC_CCS Category                    3897 non-null   object
 4   TSC_CCS Title                       3897 non-null   object
 5   TSC_CCS Description                 3897 non-null   object
 6   Proficiency Level                   3897 non-null   object
 7   Proficiency Description             3897 non-null   object
 8   Knowledge / Ability Classification  3897 non-null   object
 9   Knowledge / Ability Items           3897 non-null   object
dtypes: object(10)
memory usage: 304.6+ KB


In [13]:
pe_kb = pe_knowledge_df.groupby(['TSC_CCS Title', 'Proficiency Level'], as_index=False).agg({'Knowledge / Ability Items': ', '.join})

In [14]:
pe_kb

,TSC_CCS Title,Proficiency Level,Knowledge / Ability Items
0,Additive Manufacturing,2,"Principles of product, parts and/or component ..."
1,Additive Manufacturing,3,"Customer requirements, Principles of precision..."
2,Additive Manufacturing,4,"AM characteristics, techniques and systems, Re..."
3,Additive Manufacturing,5,"Applications of emerging AM technologies, Impa..."
4,Additive Manufacturing,6,"Parameters and rules for AM, Principles of pro..."
...,...,...,...
287,Workplace Safety and Health Practice,3,Types of relevant WSH legislations and industr...
288,Workplace Safety and Health Practice,4,"Types of WSH management programmes, Elements o..."
289,Workplace Safety and Health System Management,4,Organisational policies and procedures relatin...
290,Workplace Safety and Health System Management,5,"Workplace procedures for reporting WSH issues,..."


# Pre-processing (PE Courses)

In [15]:
# Read Course information
pe_course_raw_data = pd.read_excel(course_raw_data_path, sheet_name=course_raw_data_sheet)
pe_course_raw_data = pe_course_raw_data.drop(columns=course_raw_data_cols)
pe_course_raw_data.head(2)

,TSC Category,TSC Title/Skill,Course Title,Course Reference Number,Training Provider,WSQ/Non-WSQ,Appears in 11K list?
0,Automation Management,Automated Operation Monitoring,E-Equip-Fab Autom. Proj B (SF),TGS-2018501623,GLOBALFOUNDRIES SINGAPORE PTE. LTD.,SSG-WSQ,N
1,Automation Management,Automated Operation Monitoring,E-Equip-Fab Autom. Proj D (SF),TGS-2018501625,GLOBALFOUNDRIES SINGAPORE PTE. LTD.,SSG-WSQ,N


In [16]:
pe_course_raw_data.columns

Index(['TSC Category', 'TSC Title/Skill', 'Course Title',
       'Course Reference Number', 'Training Provider', 'WSQ/Non-WSQ',
       'Appears in 11K list?'],
      dtype='object')

In [17]:
# Read Course description
pe_course_description_raw_data = pd.read_excel(course_descr_data_path)

In [18]:
# Drop rows without information
pe_course_description_raw_data = pe_course_description_raw_data.dropna()
pe_course_description_raw_data = pe_course_description_raw_data.reset_index(drop=True)
pe_course_description_raw_data.head(2)

,Course Title,Course Reference Number,TP Name,TP UEN,Programme Type,About This Course,What You'll Learn
0,Food Safety Course Level 1,TGS-2018502008,COLD STORAGE SINGAPORE (1983) PTE LTD,194700005R,SSG-WSQ,"On completion, participants will have the skil...",1. Practise Good Food Hygiene_x000D_\n2. Use S...
1,Food Safety Course Level 1 Refresher,TGS-2018503461,COLD STORAGE SINGAPORE (1983) PTE LTD,194700005R,SSG-WSQ,"On completion, participants will have the skil...",1. Practise Good Food Hygiene_x000D_\n2. Use S...


In [19]:
#Merge both course information with the same course reference number
merged_course_df = pd.merge(pe_course_description_raw_data, pe_course_raw_data[['TSC Category', 'TSC Title/Skill','Course Reference Number', 'Training Provider', 'WSQ/Non-WSQ','Appears in 11K list?']], on='Course Reference Number', how='inner')
merged_course_df.head(2)

,Course Title,Course Reference Number,TP Name,TP UEN,Programme Type,About This Course,What You'll Learn,TSC Category,TSC Title/Skill,Training Provider,WSQ/Non-WSQ,Appears in 11K list?
0,Food Safety Course Level 1,TGS-2018502008,COLD STORAGE SINGAPORE (1983) PTE LTD,194700005R,SSG-WSQ,"On completion, participants will have the skil...",1. Practise Good Food Hygiene_x000D_\n2. Use S...,Manufacturing and Operations,Cleanliness and Contamination Control,COLD STORAGE SINGAPORE (1983) PTE LTD,SSG-WSQ,N
1,Food Safety Course Level 1 Refresher,TGS-2018503461,COLD STORAGE SINGAPORE (1983) PTE LTD,194700005R,SSG-WSQ,"On completion, participants will have the skil...",1. Practise Good Food Hygiene_x000D_\n2. Use S...,Manufacturing and Operations,Cleanliness and Contamination Control,COLD STORAGE SINGAPORE (1983) PTE LTD,SSG-WSQ,N


In [20]:
merged_course_df["course_text"] = merged_course_df['Course Title'] + " |: " + merged_course_df["About This Course"] + " | " + merged_course_df["What You'll Learn"]
merged_course_df['unique_text'] = merged_course_df['course_text'] + merged_course_df['TSC Title/Skill']
merged_course_df['unique_id'] = merged_course_df['unique_text'].apply(lambda x: generate_hash(x))

In [21]:
merged_course_df.head(2)

,Course Title,Course Reference Number,TP Name,TP UEN,Programme Type,About This Course,What You'll Learn,TSC Category,TSC Title/Skill,Training Provider,WSQ/Non-WSQ,Appears in 11K list?,course_text,unique_text,unique_id
0,Food Safety Course Level 1,TGS-2018502008,COLD STORAGE SINGAPORE (1983) PTE LTD,194700005R,SSG-WSQ,"On completion, participants will have the skil...",1. Practise Good Food Hygiene_x000D_\n2. Use S...,Manufacturing and Operations,Cleanliness and Contamination Control,COLD STORAGE SINGAPORE (1983) PTE LTD,SSG-WSQ,N,"Food Safety Course Level 1 |: On completion, p...","Food Safety Course Level 1 |: On completion, p...",13ff8cfaf8d70c1f4d585586a1809f6d310840f8fc5511...
1,Food Safety Course Level 1 Refresher,TGS-2018503461,COLD STORAGE SINGAPORE (1983) PTE LTD,194700005R,SSG-WSQ,"On completion, participants will have the skil...",1. Practise Good Food Hygiene_x000D_\n2. Use S...,Manufacturing and Operations,Cleanliness and Contamination Control,COLD STORAGE SINGAPORE (1983) PTE LTD,SSG-WSQ,N,Food Safety Course Level 1 Refresher |: On com...,Food Safety Course Level 1 Refresher |: On com...,bb1d91e54518c03c97ced4b727da678a2ec2ac3cad0d78...


# ARC Chart Reference

In [26]:
target_df.shape

(2372, 18)

In [28]:
target_df.head(2)

,Course Title,Course Reference Number,TP Name,TP UEN,Programme Type,About This Course,What You'll Learn,TSC Category,TSC Title/Skill,Training Provider,WSQ/Non-WSQ,Appears in 11K list?,proficiency_level,reason,confidence,course_text,unique_text,unique_id
0,Food Safety Course Level 1,TGS-2018502008,COLD STORAGE SINGAPORE (1983) PTE LTD,194700005R,SSG-WSQ,"On completion, participants will have the skil...",1. Practise Good Food Hygiene_x000D_\n2. Use S...,Manufacturing and Operations,Cleanliness and Contamination Control,COLD STORAGE SINGAPORE (1983) PTE LTD,SSG-WSQ,N,0,Course covers food safety which is distinct fr...,high,"Food Safety Course Level 1 |: On completion, p...","Food Safety Course Level 1 |: On completion, p...",13ff8cfaf8d70c1f4d585586a1809f6d310840f8fc5511...
2,Food Safety Course Level 1,TGS-2019504792,GOODWOOD PARK HOTEL PRIVATE LIMITED,194700135H,SSG-WSQ,To equip food handlers with the knowledge and ...,1. Practice good personal hygiene_x000D_\n2. U...,Workplace Safety and Health Management,Workplace Safety and Health System Management,GOODWOOD PARK HOTEL PRIVATE LIMITED,SSG-WSQ,N,0,"The course is related to food safety, not Work...",high,Food Safety Course Level 1 |: To equip food ha...,Food Safety Course Level 1 |: To equip food ha...,fb2a69f7b9e72b1513b40ac673a454a5950d60f3a4b263...


In [24]:
pe_kb_dic = pe_knowledge_df.groupby('TSC_CCS Title').apply(lambda x: x.groupby('Proficiency Level')['Knowledge / Ability Items'].apply(', '.join).reset_index().to_dict(orient="records"))

/tmp/ipykernel_654/2208241152.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  pe_kb_dic = pe_knowledge_df.groupby('TSC_CCS Title').apply(lambda x: x.groupby('Proficiency Level')['Knowledge / Ability Items'].apply(', '.join).reset_index().to_dict(orient="records"))


In [25]:
id_list = []
result_list = []

In [48]:
id_list, result_list = get_result(
    df=target_df,
    max_worker=20,
    id_list=id_list,
    pe_kb_dic=pe_kb_dic,
    result_list=result_list,
    checkpoint_filename=checkpoint_file_path
)

...


In [49]:
result_list[0], len(id_list)

({'level': 2,
  'reason': 'The training course content focuses on basic and fundamental aspects of Workplace Safety and Health (WSH) practice such as contributing to WSH programs, responding to emergencies, preparing for safe work activities, identifying and following WSH procedures, and re-instating work areas and equipment. These tasks align closely with the performance expectations at Proficiency Level 2, which include conducting WSH checks, identifying hazards, and carrying out risk controls in accordance with organizational procedures and legislative requirements.',
  'confidence': 'high'},
 7101)

In [51]:
# with open(pickle_file_path, "wb") as file:
#     pkl.dump((id_list, result_list), file)

In [30]:
with open(pickle_file_path, "rb") as file:
    id_list, result_list = pkl.load(file)

In [31]:
result_df = pd.DataFrame(data={"unique_id": id_list, "proficiency_level": [x['level'] for x in result_list], "reason": [x['reason'] for x in result_list], "confidence": [x['confidence'] for x in result_list]})

In [34]:
sub_result_df = result_df[result_df.unique_id.isin(list(target_df.unique_id))]
sub_result_df.head(2)

,unique_id,proficiency_level,reason,confidence
1,36122cce2304baaec24a24a089d82ed7228d80ac1daca0...,1,The training course content for Computer Aided...,high
3,4f1429d869f48a91e5040ceaae5904538a92052a0d2486...,3,The training course content 'Statistical Proce...,high


In [47]:
sub_result_df.rename(columns={"proficiency_level": "proficiency_level_rac_chart", "reason": "reason_rac_chart", "confidence": "confidence_rac_chart"}, inplace=True)

/tmp/ipykernel_654/1021841814.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_result_df.rename(columns={"proficiency_level": "proficiency_level_rac_chart", "reason": "reason_rac_chart", "confidence": "confidence_rac_chart"}, inplace=True)


In [48]:
final_df = data.merge(sub_result_df, how='left', on='unique_id')

In [35]:
sub_result_df.shape

(2372, 4)

In [41]:
data.shape

(6950, 18)

In [50]:
final_df.drop(columns=['course_text','unique_text','unique_id'], inplace=True)

In [51]:
final_df.to_csv(final_output_path, index=False)

# Poor Data Quality Courses

In [60]:
original_pe_crs_descr_df = pd.read_excel(course_descr_data_path)

In [61]:
merged_raw_course_df = pd.merge(original_pe_crs_descr_df, pe_course_raw_data[['TSC Category', 'TSC Title/Skill','Course Reference Number', 'Training Provider', 'WSQ/Non-WSQ','Appears in 11K list?']], on='Course Reference Number', how='inner')

In [62]:
poor_dq_crs = merged_raw_course_df[~merged_raw_course_df['Course Reference Number'].isin(list(final_df['Course Reference Number']))]

In [63]:
poor_dq_crs['Course Reference Number'].nunique()

489

In [65]:
poor_dq_crs.info()

<class 'pandas.core.frame.DataFrame'>
Index: 635 entries, 0 to 7549
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Course Title             162 non-null    object
 1   Course Reference Number  635 non-null    object
 2   TP Name                  162 non-null    object
 3   TP UEN                   162 non-null    object
 4   Programme Type           162 non-null    object
 5   About This Course        154 non-null    object
 6   What You'll Learn        159 non-null    object
 7   TSC Category             635 non-null    object
 8   TSC Title/Skill          635 non-null    object
 9   Training Provider        635 non-null    object
 10  WSQ/Non-WSQ              635 non-null    object
 11  Appears in 11K list?     635 non-null    object
dtypes: object(12)
memory usage: 64.5+ KB


In [74]:
missing_content_df = poor_dq_crs[poor_dq_crs['Course Title'].isnull()]

In [76]:
missing_content_df.shape

(473, 12)

In [78]:
remaining_poor_dq_df = poor_dq_crs[~poor_dq_crs['Course Reference Number'].isin(missing_content_df['Course Reference Number'])]

In [80]:
remaining_poor_dq_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 162 entries, 499 to 7549
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Course Title             162 non-null    object
 1   Course Reference Number  162 non-null    object
 2   TP Name                  162 non-null    object
 3   TP UEN                   162 non-null    object
 4   Programme Type           162 non-null    object
 5   About This Course        154 non-null    object
 6   What You'll Learn        159 non-null    object
 7   TSC Category             162 non-null    object
 8   TSC Title/Skill          162 non-null    object
 9   Training Provider        162 non-null    object
 10  WSQ/Non-WSQ              162 non-null    object
 11  Appears in 11K list?     162 non-null    object
dtypes: object(12)
memory usage: 16.5+ KB


In [81]:
remaining_poor_dq_df

,Course Title,Course Reference Number,TP Name,TP UEN,Programme Type,About This Course,What You'll Learn,TSC Category,TSC Title/Skill,Training Provider,WSQ/Non-WSQ,Appears in 11K list?
499,Comply with Workplace Safety and Health Polici...,TGS-2012500966,CBM PTE. LTD.,197100922N,SSG-WSQ,to provide participants with a competency-base...,- Assessment Plan_x000D_\n- Assessment Marking...,Workplace Safety and Health Management,Workplace Safety and Health Practice,CBM PTE. LTD.,SSG-WSQ,N
500,Comply with Workplace Safety and Health Polici...,TGS-2012500966,CBM PTE. LTD.,197100922N,SSG-WSQ,to provide participants with a competency-base...,- Assessment Plan_x000D_\n- Assessment Marking...,Workplace Safety and Health Management,Workplace Safety and Health System Management,CBM PTE. LTD.,SSG-WSQ,N
549,Robotics Programming (ROS) for Advanced Manufa...,TGS-2023021860,BOSCH REXROTH PTE. LTD.,197700888D,SSG-NON-WSQ,Participants will gain an understanding on Rob...,Robotics programming with Robot Operating Syst...,Manufacturing and Operations,Manufacturing Technology,BOSCH REXROTH PTE. LTD.,SSG-NON-WSQ,N
574,Supervise Workplace Safety and Health in Proce...,TGS-2013500766,PEC LTD.,198200079M,SSG-WSQ,"On completion of this unit, learners will have...",CE1. Comply with workplace safety and health l...,Workplace Safety and Health Management,Workplace Safety and Health Practice,PEC LTD.,SSG-WSQ,N
575,Supervise Workplace Safety and Health in Proce...,TGS-2013500766,PEC LTD.,198200079M,SSG-WSQ,"On completion of this unit, learners will have...",CE1. Comply with workplace safety and health l...,Workplace Safety and Health Management,Workplace Safety and Health System Management,PEC LTD.,SSG-WSQ,N
...,...,...,...,...,...,...,...,...,...,...,...,...
7516,Industrial IoT Analytics (in MC in Industrial ...,TGS-2021008869,Temasek Polytechnic,T08GB0062L,SSG-NON-WSQ,This subject provides the essential concepts a...,The students will learn to develop dashboard f...,Manufacturing and Operations,Manufacturing Technology,Temasek Polytechnic,SSG-NON-WSQ,N
7517,Introduction to Design Thinking (in MC in Desi...,TGS-2021008873,Temasek Polytechnic,T08GB0062L,SSG-NON-WSQ,This is a foundational subject that introduces...,It will include several design research method...,Network Technology Management,User Experience Design,Temasek Polytechnic,SSG-NON-WSQ,N
7518,Design Thinking in Practice (in MC in Design T...,TGS-2021008874,Temasek Polytechnic,T08GB0062L,SSG-NON-WSQ,Student will be exposed to ideation tool such ...,This is a â€œClose to Real Worldâ€ project-ba...,Precision Manufacturing Process,Additive Manufacturing,Temasek Polytechnic,SSG-NON-WSQ,N
7548,"Gamification in Healthcare driven by AI, IOT a...",TGS-2023025968,Temasek Polytechnic,T08GB0062L,SSG-NON-WSQ,This comprehensive course is designed to furni...,â€¢Explain the use of game technology in healt...,Business and Organisational Management,Networking,Temasek Polytechnic,SSG-NON-WSQ,N


In [82]:
[x for x in remaining_poor_dq_df['Course Reference Number'] if x in final_df['Course Reference Number']]

[]

In [83]:
missing_content_df.to_csv("../round_2_output/missing_content_course.csv", index=False)
remaining_poor_dq_df.to_csv("../round_2_output/poor_content_quality_course.csv", index=False)